## Project: Data querying and analysis using NoSQL Database

## Problem Statement

Using the Cassandra NoSQL database, explore the **Movielens** dataset and build a movie recommendation engine

## Learning Objectives

At the end of the mini-project, we will be able to :

* understand the concepts of NoSQL platforms
* implement the database connectivity using Cassandra
* create and insert data to the database 
* perform querying, retrieving and analyzing the data from database

### Dataset

The dataset chosen for this project is [Movielens Dataset](https://movielens.org/). The dataset describes movies and ratings from Movielens. It contains movies from year 1919 to 2000 of total 3883 movies. There are 6040 users rated one or more than one movie of total 1000209 ratings collected. Movielens is maintained by members of GroupLens Research at the University of Minnesota. There are 3 datasets given, and they are:

1. `data_movie.csv` that contains movies information:

* MovieId
* Title
* Genres

2. `data_rating.csv` that contains ratings of movies by users:

* UserId
* MovieId
* Rating

3. `data_users.csv` that contains users information:

* UserId
* Age
* Gender

## Information

#### NoSQL platforms

A NoSQL database provides a mechanism for storage and retrieval of data that is modeled in means other than the tabular relations used in relational databases.

**Cassandra** 

Apache Cassandra is a free and open-source, distributed, wide-column store, NoSQL database management system designed to handle large amounts of data across many commodity servers, providing high availability with no single point of failure. It is a NOSQL database developed by Facebook. It is a great database that allows you to effectively run queries on large amount of structured and semi structured data.

To know when you have to choose Cassandra as your database of choice, you need to have an understanding of CAP Theorem.

CAP theorem states that it is impossible for a distributed system to satisfy all three of these guarantees together, you need to leave one out.

- C is Consistency, means all nodes have the same data at the same time
- P is partition tolerance, means that in case of failure the system continues to operate
- A is availability , make sure every request receives a response

![img](https://developer.ibm.com/developer/default/tutorials/ba-set-up-apache-cassandra-architecture/images/figure1.png)

### Components of Cassandra

There are following components in the Cassandra

![img](https://www.guru99.com/images/cassandra/021116_0524_CassandraAr1.png)

**Node**
Node is the place where data is stored. It is the basic component of Cassandra.

**Data Center**
A collection of nodes are called data center. Many nodes are categorized as a data center.

**Cluster**
The cluster is the collection of many data centers.

**Commit Log**
Every write operation is written to Commit Log. Commit log is used for crash recovery.

**Mem-table**
After data written in Commit log, data is written in Mem-table. Data is written in Mem-table temporarily.

**SSTable**
When Mem-table reaches a certain threshold, data is flushed to an SSTable disk file.

### Data Replication

As hardware problem can occur or link can be down at any time during data process, a solution is required to provide a backup when the problem has occurred. So data is replicated for assuring no single point of failure.

Cassandra places replicas of data on different nodes based on these two factors.
- Where to place next replica is determined by the Replication Strategy.
- While the total number of replicas placed on different nodes is determined by the Replication Factor.

One Replication factor means that there is only a single copy of data while three replication factor means that there are three copies of the data on three different nodes.

**Install the cassandra-driver**

In [1]:
!pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.2 MB/s eta 0:00:00


#### Importing Necessary Packages

In [2]:
import pandas as pd
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from matplotlib import pyplot as plt

In [3]:
print(cassandra.__version__)

3.25.0


### Database connection 

Astra Datastax is used to create a database


**Connect the database and create keyspace:**

* Download Secure Connect Bundle zip file from Datastax section. Follow the instructions on the page
* Upload the `Secure-connect-XXXX.zip` file, which is downloaded from datastax.
* Generate the token and save the credentials from settings section.
    - Hint: Select role as `admin-user` and generate token
* Using the credentials generated in settings, specify the `client Id` and `Client Secret` to the variables below.

In [ ]:
#@title Download bundle zip
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/secure-connect-cds.zip

In [ ]:
# from google.colab import files
# files.upload()

set the `Secure connect bundle zip file` path and specify the `clientID` and `Client_Secret`



In [4]:
zip_path = '/content/secure-connect-cds.zip'
Client_ID = 'SzdMZDsXLvXUQiHRsEogQgtR'
Client_Secret = 'SaYcoWaejFAx4CxXzuf1spOMRa+t1oyTd8Z+Medbuba1q0Ww5AY,1MOPNvrr9wWSnR82,IiQa4muFoF8OfOhxdndNXtZbuZsv.dSwKKccUaHr96B8-88gyAWGURFO2Wa'

#### Create a Cluster instance to connect to your Astra database.

You will typically have one instance of Cluster for each Cassandra cluster you want to interact with. Create a session object using the cluster.



In [ ]:
cloud_config= {
        'secure_connect_bundle': zip_path
}
auth_provider = PlainTextAuthProvider(Client_ID, 
                                      Client_Secret)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

#### Verifying the database connection

execute a simple query and verify the database connection

In [ ]:
row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

#### Setting the Key Space

A keyspace is the top-level database object that controls the replication for the object it contains at each datacenter in the cluster. Keyspaces contain tables, materialized views and user-defined types, functions and aggregates. Typically, a cluster has one keyspace per application. Since replication is controlled on a per-keyspace basis, store data with different replication requirements (at the same datacenter) in different keyspaces.

Before creating tables and inserting data let's create and set the keyspace

* we can create keyspace manually on Datastax dashboard or also using the CQL command. (use lowercase to avoid errors)

    * [Hint](https://docs.datastax.com/en/cql-oss/3.x/cql/cql_reference/cqlCreateKeyspace.html)

* once the keyspace is created successfully, set the keyspace using the command `set_keyspace()`

In [ ]:
try:
    session.set_keyspace('mov')
except Exception as e:
    print(e)

### Insert the data into Database 

To insert the data into database, we can create a table in keyspace and insert the data by using CQL command or we can directly upload the data on the datstax dashboard using Astra Dataloader

As there is huge amount of data, insertion of data using CQL commands may take time, preferable method is Astra Dataloader.

* Download the below mentioned 3 datasets and upload to Datastax Astra dashboard using Astra Dataloader
    * [data_movies](https://cdn.iisc.talentsprint.com/CDS/MiniProjects/data_movies.csv)
    * [data_ratings](https://cdn.iisc.talentsprint.com/CDS/MiniProjects/data_ratings.csv)
    * [data_users](https://cdn.iisc.talentsprint.com/CDS/MiniProjects/data_users.csv)
    
while uploading data using Datastax Astra dataloader you may encounter the below error message due to formatting issues or bulk data. In that case, click on upload button for second time for data upload. 
![img](https://cdn.iisc.talentsprint.com/CDS/MiniProjects/data_upload_error.JPG)

Note: Cassandra also handles non-SQL data formats such as JSON (see [insert JSON data](https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertJSON.html)). However for the purpose of this Mini-project, we are using a tabular data format (.csv).

### Querying the database

* Select First 10 rows of the data_movies table
* How many Users have watched the movie  `'Contact (1997)'` with MovieId = 1584
* Select the count of teenage male users




In [ ]:
query = "SELECT * FROM mov.data_movies limit 10;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

In [ ]:
rating_query = 'select count(*) from data_ratings where movieid=1584 allow filtering;'
try:
    rows = session.execute(rating_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

In [ ]:
user_query = """select count(*) from data_users  where age<20 and gender='M' allow filtering;"""
try:
    rows = session.execute(user_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

* Select the count of movies with Top rating from the entire ratings table

In [ ]:
query = "select count(*) from mov.data_ratings where rating=5 allow filtering;"
try:
    top_rated_count = session.execute(query)
    for c in top_rated_count:
        print(c)
except Exception as e:
    print(e)

* Select the count of movies, ONLY in Comedy genre from the entire table

In [ ]:
Query = 'SELECT COUNT(*) FROM mov.data_movies WHERE genres={} ALLOW FILTERING;'.format("'Comedy'")
print(Query)
try:
    count = session.execute(Query)
    for i in count:
        print(i)
except Exception as e:
    print(e)

### Visualizing the Data

Retrieve the data using CQL queries and visualize using matplotlib

* Visualize the frequencies of movie genres

In [ ]:
query = 'select genres from mov.data_movies'
genres = []
try:
    top_rated_count = session.execute(query)
    for c in top_rated_count:
      #print(c.genres)
      genres.extend(c.genres.split("|"))
except Exception as e:
    print(e)

In [ ]:
plt.hist(genres);
plt.xticks(rotation=45)
plt.show()

* Visualize the bar plot showing no. of movies released in a particular year

In [ ]:
query = "select * from mov.data_movies"
century = []
not_centuries = []
try:
    top_rated_count = session.execute(query)
    for c in top_rated_count:
        if c.title.find("("):
            century.append(int(c.title[-5:-1]))
        else:
            not_centuries.append(c.title)
except Exception as e:
    print(e)

In [ ]:
century = sorted(century)
pd.DataFrame(century).value_counts(sort=False).plot.bar();
plt.xticks(rotation=45)
plt.show()

* Depict the age group that has the maximum viewership of movies



In [ ]:
query = 'select age from mov.data_users;'
Ages=[]
try:
    result_set = session.execute(query)
    for i in result_set:
        Ages.append((i.age//10 ) * 10)
except Exception as e:
    print(e)

In [ ]:
from matplotlib import pyplot as plt
plt.hist(Ages);
plt.xticks(rotation=45)
plt.show()

### Analyzing the Data

Analyze the correlation between the movies records and implement a function to recommend movies based on the search term

* Retrieve all the data from ratings table

In [ ]:
query = 'select * from mov.data_ratings;'
movie_ratings = []
try:
    res_set = session.execute(query)
    for c in res_set:
        movie_ratings.append([c.movieid,c.rating,c.userid])
except Exception as e:
    print(e)

* Create a movie matrix by applying pivot table with the above retrieved ratings data

* use `pd.pivot_table` with index = `userid`, columns = `movieid` and Values = `rating`

 **Hint:** create a dataframe with above retrieved ratings data to apply pivot table

In [ ]:
movie_ratings = pd.DataFrame(movie_ratings,columns=['MovieId','Rating','UserId'])
movie_ratings.head(2)

In [ ]:
movie_matrix = movie_ratings.pivot_table(index = ['UserId'],columns = ["MovieId"],values = "Rating").fillna(0)

* Define a function named `recommend_movies()` which will take a movieid as argument and return the top 5 similar movieids

    * Considering the argument (movieid) as search item collect the particular row data
    
    * Using movie matrix calculate correlation between movie search Item data and all the remaining data
        
    * Sort the result by ascending order and return the top 5 items

In [ ]:
def recommend_movie(movie):
    movie_watched = movie_matrix[movie]
    similarity_with_other_movies = movie_matrix.corrwith(movie_watched)
    similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
    return similarity_with_other_movies.head()

* write a query to get MovieId of a Title (your choice) and the result MovieId as a search item by calling the above defined function

In [ ]:
query = 'select * from mov.data_movies where title={} allow filtering;'.format("'Boy Who Could Fly, The (1986)'")
print(query)
try:
    res_set = session.execute(query)
    for i in res_set:
        print("Movie id is",i.movieid)
except Exception as e:
    print(e)

In [ ]:
similar = recommend_movie(2453)
similar.index[1:]

* Now taking the result of top 5 items (MovieId's) write a query to retrieve the Movie title from movies table

In [ ]:
for idx in similar.index[1:]:
  query = 'select * from data_movies where movieid= {}'.format(idx)
  try:
      res_set = session.execute(query)
      for i in res_set:
          print(i.title, i.genres)
  except Exception as e:
      print(e)

**Close the session**

In [ ]:
session.shutdown()
cluster.shutdown()